In [14]:
import keras
from keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape
#from keras.layers import BatchNormalization
from keras.models import Model
from keras import backend as K
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [15]:
from DataLoader import DataLoader
from surprise import Dataset

dataset = DataLoader.normalizedRatingsToTensor(Dataset.load_builtin().build_full_trainset())
np_dataset = dataset.numpy()
dataset = tf.convert_to_tensor(np_dataset)

In [16]:
encoder_input = Input(dataset.shape[1], name='encoder_input')
x = Dense(1000, activation='tanh')(encoder_input)
x = Dense(500, activation='tanh')(x)
x = Dense(200, activation='tanh')(x)

In [17]:
latent_dim = 100

z_mu = Dense(latent_dim, name="latent_mu")(x)
z_sigma = Dense(latent_dim, name='latent_sigma')(x)

In [18]:
def reparametrize(args):
    mu, sigma = args
    eps = K.random_normal(shape=K.shape(mu))
    return mu + K.exp(sigma / 2) * eps

In [19]:
z = Lambda(reparametrize, output_shape=(latent_dim, ), name='z')([z_mu, z_sigma])

In [20]:
encoder = Model(encoder_input, [z_mu, z_sigma, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 1682)]       0           []                               
                                                                                                  
 dense_6 (Dense)                (None, 1000)         1683000     ['encoder_input[0][0]']          
                                                                                                  
 dense_7 (Dense)                (None, 500)          500500      ['dense_6[0][0]']                
                                                                                                  
 dense_8 (Dense)                (None, 200)          100200      ['dense_7[0][0]']                
                                                                                            

In [21]:
decoder_input = Input(shape=(latent_dim), name='decoder_input')
x = Dense(200, activation='tanh')(decoder_input)
x = Dense(500, activation='tanh')(x)
x = Dense(1000, activation='tanh')(x)
x = Dense(dataset.shape[1], activation="sigmoid", name='decoder_output')(x)
decoder = Model(decoder_input, x, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 100)]             0         
                                                                 
 dense_9 (Dense)             (None, 200)               20200     
                                                                 
 dense_10 (Dense)            (None, 500)               100500    
                                                                 
 dense_11 (Dense)            (None, 1000)              501000    
                                                                 
 decoder_output (Dense)      (None, 1682)              1683682   
                                                                 
Total params: 2,305,382
Trainable params: 2,305,382
Non-trainable params: 0
_________________________________________________________________


In [22]:
decoder_output = decoder(z)

In [23]:
class CustomLayer(keras.layers.Layer):

    def loss(self,x,z_decoded):
        recon_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * K.mean(1 + z_sigma - K.square(z_mu) - K.exp(z_sigma), axis=-1)

        return K.mean(recon_loss + kl_loss)
    
    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        return x

In [24]:
y = CustomLayer()([encoder_input, decoder_output])

In [25]:
vae = Model(encoder_input, y, name='vae')
vae.compile(optimizer='adam', loss=None)
vae.summary()

Model: "vae"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 1682)]       0           []                               
                                                                                                  
 dense_6 (Dense)                (None, 1000)         1683000     ['encoder_input[0][0]']          
                                                                                                  
 dense_7 (Dense)                (None, 500)          500500      ['dense_6[0][0]']                
                                                                                                  
 dense_8 (Dense)                (None, 200)          100200      ['dense_7[0][0]']                
                                                                                                

In [26]:
vae.fit(dataset, None, epochs=10, batch_size=128)

Epoch 1/10


TypeError: in user code:

    File "/home/nikola/projects/master-rad/env/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/nikola/projects/master-rad/env/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/nikola/projects/master-rad/env/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/nikola/projects/master-rad/env/lib/python3.8/site-packages/keras/engine/training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/nikola/projects/master-rad/env/lib/python3.8/site-packages/keras/engine/training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "/home/nikola/projects/master-rad/env/lib/python3.8/site-pa